# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて、言語モデルを用いた「20の質問」ゲームをプレイするためのエージェントの作成に取り組んでいます。具体的には、以下の手法やライブラリを使用して問題を解決しています。

### 取り組む問題
ノートブックでは、特定のキーワード（国や物の名前など）を推測するゲームを自動化するAIエージェントを開発しています。エージェントは、質問を生成したり、はい・いいえの質問に対する回答を生成したり、最終的にキーワードを推測する役割を担います。

### 使用しているライブラリ
- **Transformers**: Hugging Faceのライブラリを使用して、事前訓練済みの言語モデル（Causal Language Model）を扱います。
- **Torch**: PyTorchを使用して、GPUを活用しつつメモリ効率の良いモデルの操作を行います。
- **BitsAndBytes**: モデルの量子化を行い、8ビットでの読み込みをサポートします。

### 実装内容
1. **環境の設定**: 必要なライブラリ（transformers, bitsandbytes）をインストールし、ファイルの準備を行います。
2. **モデルの定義**: 事前訓練されたモデルを特定のGPUに配置してメモリ効率を最適化し、量子化の設定も行います。
3. **質問および回答の生成**: ユーザーインターフェースを介して、質問と回答を生成するメソッド（asker、answerer）を持つ`Robot`クラスを実装します。
4. **エージェントのメインロジック**: `agent`関数を通じて、推測、質問、および回答のターンを処理します。もし応答が不適切な場合はデフォルトで「yes」を返すロジックも含まれています。
5. **キーワードの準備**: JSON形式のデータからキーワードとそのカテゴリを読み込み、データフレームに変換します。
6. **ゲームの実行**: 20ラウンドにわたり、エージェントが質問を生成し、回答を受け取って推測を行う様子をシミュレーションします。

このノートブックを通じて、特定のキーワードを推測するためのAIエージェントが、生成的な質問を通じて効率的に情報を引き出すことができるプロセスを示しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語について簡単に解説します。対象は、ノートブックの内容に関連する可能性があるマイナーな用語や、実務経験がないと馴染みが薄い用語です。

1. **メモリ効率的なSDP (Mem Efficient SDP)**:
   - SDP（Stochastic Dynamic Programming）は、強化学習などで使用される手法ですが、メモリ効率的なSDPは、特に大規模なモデルを扱う際にメモリ使用量を削減する手法です。これにより、より少ないリソースでモデルの学習や推論を実施できます。

2. **BitsAndBytesConfig**:
   - このクラスは、モデルの量子化（特に8ビット量子化など）を設定するためのものです。量子化は、モデルのサイズを縮小し、計算を高速化するために重みを圧縮する手法です。

3. **Gradient Checkpointing (勾配チェックポイント)**:
   - 大規模モデルの訓練でメモリを節約するためのテクニックです。全ての中間出力を保存するのではなく、効率的に再計算することでメモリを削減します。これにより、より大きなバッチサイズやより深いネットワークを扱うことが可能になります。

4. **デバイスマップ (Device Map)**:
   - テンソルをどのGPUに配置するかを定義するためのマッピングです。特に、複数のGPUを利用する際に、各レイヤーをどのGPUに配置するかを示します。

5. **トークナイザー (Tokenizer)**:
   - テキストをモデルが理解できる形式に変換するためのコンポーネントです。単語やサブワードをトークンと呼ばれる数値に変換します。通常、自然言語処理の前処理として重要です。

6. **終了トークン (End Of Sentence Token, EOS)**:
   - 生成されたテキストの文の終わりを示すために使用される特別なトークンです。生成モデルが出力を停止するタイミングを決定するのに用いられます。

7. **観察オブジェクト (Observation Object)**:
   - 現在のゲームの状態やターンセッティングなど、エージェントが行動を決定するために必要な情報を保持するためのクラスです。ゲームにおける状態情報を管理します。

8. **代替キーワード (Alternative Keywords)**:
   - 特定のキーワードに関連する、他の言葉やフレーズを指します。モデルが完全に同じ意味の異なる表現を理解できるようにするために用いられます。

9. **ランダム選択 (Random Choice)**:
   - プログラムがプレイヤーの質問や答える際に、あらかじめ設定した選択肢から無作為に選ぶ手法です。これは、自然かつ多様なインタラクションを生むために使われます。

これらの用語は、ノートブックの内容を理解する際に初学者が直面しやすい概念でもありますので、あらかじめ知識を持っておくと良いでしょう。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers
```

</div>
</details>

In [ ]:
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
#else:
model_id = "/kaggle/input/gemma/transformers/7b-it/3"

device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3") 
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

device = {layer:gpu_mem for (layer,gpu_mem) in device_maps}
config = AutoConfig.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
config.gradient_checkpointing = True
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3", torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

def generate_answer(template):
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    start_gen = input_ids.input_ids.shape[1]
    output_ids = output_ids[start_gen:]
    if id_eot in output_ids:
        stop = output_ids.tolist().index(id_eot)
        output = tokenizer.decode(output_ids[:stop])
    else:
        output = tokenizer.decode(output_ids)
    output = re.sub('\n', '', output)
    output = re.sub(' <end_of_turn>', '', output)
    output = re.sub('<end_of_turn>', '', output)
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "yes"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    def asker(self, obs):
        sys_prompt = """
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        The keyword is a specific country.
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            Let's play 20 Questions. You are playing the role of the Questioner.
            Please ask a yes-or-no question.
            to help you, here's an example of how it should work assuming that the keyword is Morocco:
            examle:
            <you: is it a contry?
            user: yes
            you: is it in europe?
            user: no
            you: is it in africa?
            user: yes
            you: do most people living there have dark skin?
            user: no
            you: is it a country name starting by m ?
            user: yes
            you: is it Morocco?
            user: yes.>
            the user has chosen the word, ask your first question!
            please be short and not verbose, give only one question, no extra word!
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions)>=1:
                    for q, a in zip(obs.questions, obs.answers):
                        chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                        chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
                    
        elif obs.turnType == "guess":
                conv = ""
                for q, a in zip(obs.questions, obs.answers):
                    conv += f"""Question: {q}\nAnswer: {a}\n"""
                guess_prompt =  sys_prompt + f"""
                so far, the current state of the game is as following:\n{conv}
                based on the conversation, can you guess the word, please give only the word, no verbosity around
                """
                chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
                chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    def answerer(self, obs):
        sys_prompt = f"""
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        The keyword is a specific place, or thing.\n
        so make sure you understand the user's question and you understand the keyword you're playig on.
        for now the word that the user should guess is: "{obs.keyword}", it is of category "{obs.category}",
        to help you, here's an example of how it should work assuming that the keyword is Morocco in the category "place":
        examle:
        <user: is it a place?
        you: yes
        user: is it in europe?
        you: no
        user: is it in africa?
        you: yes
        user: do most people living there have dark skin?
        you: no
        user: is it a country name starting by m ?
        you: yes
        user: is it Morocco?
        you: yes.>
        """
        chat_template = f"""<start_of_turn>system\n{sys_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "yes"
        
    return response
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
#else:
model_id = "/kaggle/input/gemma/transformers/7b-it/3"

# 各レイヤーをどのGPUに配置するかを示すマッピング
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3") 
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

# 各レイヤーを配置するGPUを辞書形式で定義
device = {layer:gpu_mem for (layer,gpu_mem) in device_maps}
config = AutoConfig.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
config.gradient_checkpointing = True
# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3", torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

# 回答を生成する関数
def generate_answer(template):
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    start_gen = input_ids.input_ids.shape[1]
    output_ids = output_ids[start_gen:]
    if id_eot in output_ids:  # 出力に終了トークンが含まれているか確認
        stop = output_ids.tolist().index(id_eot)  # 終了トークンの位置を取得
        output = tokenizer.decode(output_ids[:stop])  # トークンをデコードして出力を取得
    else:
        output = tokenizer.decode(output_ids)  # トークンをそのままデコード
    output = re.sub('\n', '', output)  # 改行を削除
    output = re.sub(' <end_of_turn>', '', output)  # 特定のトークンを削除
    output = re.sub('<end_of_turn>', '', output)  # 特定のトークンを削除
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # モードに応じた処理を実行
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():  # 出力が「yes」の場合
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():  # 出力が「no」の場合
                output = "no"
            else:
                output = "yes"  # それ以外はデフォルトで「yes」
        if mode == "guessing":
            output = self.asker(obs)  # モードが「guessing」の場合
        return output

    # 質問者の処理を行うメソッド
    def asker(self, obs):
        sys_prompt = """
        あなたは、20の質問ゲームをプレイするように設計されたAIアシスタントです。
        このゲームでは、回答者がキーワードを考え、質問者がはい・いいえの質問に答えます。
        キーワードは特定の国です。
        """
        if obs.turnType == "ask":  # 質問するターンの場合
            ask_prompt = sys_prompt + """
            20の質問をプレイしましょう。あなたは質問者の役割を果たします。
            はい・いいえの質問をしてください。
            キーワードがモロッコだとしたら、以下のように進行します:
            例:
            <あなた: それは国ですか？
            ユーザー: はい
            あなた: ヨーロッパにありますか？
            ユーザー: いいえ
            あなた: アフリカにありますか？
            ユーザー: はい
            あなた: そこに住んでいる人のほとんどは肌の色が暗いですか？
            ユーザー: いいえ
            あなた: 名前がmで始まる国ですか？
            ユーザー: はい
            あなた: モロッコですか？
            ユーザー: はい。>
            ユーザーが単語を選びました、あなたの最初の質問をしてください！
            短く、冗長にならず一つの質問だけをしてください、余分な言葉はいりません！
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions)>=1:  # 以前の質問がある場合
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                    chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        
        elif obs.turnType == "guess":  # 推測するターンの場合
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n答え: {a}\n"""
            guess_prompt =  sys_prompt + f"""
            現在のゲームの進行状況は次のようになっています:\n{conv}
            この会話に基づいて、単語を推測できますか、単語だけを回答してください、余計なことはいりません。
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    # 回答者の処理を行うメソッド
    def answerer(self, obs):
        sys_prompt = f"""
        あなたは、20の質問ゲームをプレイするように設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がはい・いいえの質問に答えます。
        キーワードは特定の場所や物です。\n
        ユーザーの質問を理解し、プレイしているキーワードを把握していることを確認してください。
        現在、ユーザーが推測すべき単語は：「{obs.keyword}」、カテゴリは「{obs.category}」です。
        もしキーワードがモロッコでカテゴリが「場所」の場合、以下のように進行します:
        例:
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: ヨーロッパにありますか？
        あなた: いいえ
        ユーザー: アフリカにありますか？
        あなた: はい
        ユーザー: そこに住んでいる人のほとんどは肌の色が暗いですか？
        あなた: いいえ
        ユーザー: 名前がmで始まる国ですか？
        あなた: はい
        ユーザー: モロッコですか？
        あなた: はい。>
        """
        chat_template = f"""<start_of_turn>system\n{sys_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        if len(obs.answers)>=1:  # 以前の回答がある場合
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

# エージェントのメイン処理
def agent(obs, cfg):
    
    if obs.turnType == "ask":  # 質問をするターンの場合
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType == "guess":  # 推測するターンの場合
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType == "answer":  # 答えるターンの場合
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:  # レスポンスが空または短い場合
        response = "yes"  # デフォルトで「yes」を返す
        
    return response
```

</div>
</details>

In [ ]:
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
#else:
model_id = "/kaggle/input/gemma/transformers/7b-it/3"

# 各レイヤーをどのGPUに配置するかを示すマッピング
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3") 
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

# 各レイヤーを配置するGPUを辞書形式で定義
device = {layer:gpu_mem for (layer,gpu_mem) in device_maps}
config = AutoConfig.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
config.gradient_checkpointing = True
# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3", torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

# 回答を生成する関数
def generate_answer(template):
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    start_gen = input_ids.input_ids.shape[1]
    output_ids = output_ids[start_gen:]
    if id_eot in output_ids:  # 出力に終了トークンが含まれているか確認
        stop = output_ids.tolist().index(id_eot)  # 終了トークンの位置を取得
        output = tokenizer.decode(output_ids[:stop])  # トークンをデコードして出力を取得
    else:
        output = tokenizer.decode(output_ids)  # トークンをそのままデコード
    output = re.sub('\n', '', output)  # 改行を削除
    output = re.sub(' <end_of_turn>', '', output)  # 特定のトークンを削除
    output = re.sub('<end_of_turn>', '', output)  # 特定のトークンを削除
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # モードに応じた処理を実行
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():  # 出力が「yes」の場合
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():  # 出力が「no」の場合
                output = "no"
            else:
                output = "yes"  # それ以外はデフォルトで「yes」
        if mode == "guessing":
            output = self.asker(obs)  # モードが「guessing」の場合
        return output

    # 質問者の処理を行うメソッド
    def asker(self, obs):
        sys_prompt = """
        あなたは、20の質問ゲームをプレイするように設計されたAIアシスタントです。
        このゲームでは、回答者がキーワードを考え、質問者がはい・いいえの質問に答えます。
        キーワードは特定の国です。
        """
        if obs.turnType == "ask":  # 質問するターンの場合
            ask_prompt = sys_prompt + """
            20の質問をプレイしましょう。あなたは質問者の役割を果たします。
            はい・いいえの質問をしてください。
            キーワードがモロッコだとしたら、以下のように進行します:
            例:
            <あなた: それは国ですか？
            ユーザー: はい
            あなた: ヨーロッパにありますか？
            ユーザー: いいえ
            あなた: アフリカにありますか？
            ユーザー: はい
            あなた: そこに住んでいる人のほとんどは肌の色が暗いですか？
            ユーザー: いいえ
            あなた: 名前がmで始まる国ですか？
            ユーザー: はい
            あなた: モロッコですか？
            ユーザー: はい。>
            ユーザーが単語を選びました、あなたの最初の質問をしてください！
            短く、冗長にならず一つの質問だけをしてください、余分な言葉はいりません！
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions)>=1:  # 以前の質問がある場合
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                    chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        
        elif obs.turnType == "guess":  # 推測するターンの場合
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n答え: {a}\n"""
            guess_prompt =  sys_prompt + f"""
            現在のゲームの進行状況は次のようになっています:\n{conv}
            この会話に基づいて、単語を推測できますか、単語だけを回答してください、余計なことはいりません。
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    # 回答者の処理を行うメソッド
    def answerer(self, obs):
        sys_prompt = f"""
        あなたは、20の質問ゲームをプレイするように設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がはい・いいえの質問に答えます。
        キーワードは特定の場所や物です。\n
        ユーザーの質問を理解し、プレイしているキーワードを把握していることを確認してください。
        現在、ユーザーが推測すべき単語は：「{obs.keyword}」、カテゴリは「{obs.category}」です。
        もしキーワードがモロッコでカテゴリが「場所」の場合、以下のように進行します:
        例:
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: ヨーロッパにありますか？
        あなた: いいえ
        ユーザー: アフリカにありますか？
        あなた: はい
        ユーザー: そこに住んでいる人のほとんどは肌の色が暗いですか？
        あなた: いいえ
        ユーザー: 名前がmで始まる国ですか？
        あなた: はい
        ユーザー: モロッコですか？
        あなた: はい。>
        """
        chat_template = f"""<start_of_turn>system\n{sys_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        if len(obs.answers)>=1:  # 以前の回答がある場合
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

# エージェントのメイン処理
def agent(obs, cfg):
    
    if obs.turnType == "ask":  # 質問をするターンの場合
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType == "guess":  # 推測するターンの場合
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType == "answer":  # 答えるターンの場合
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:  # レスポンスが空または短い場合
        response = "yes"  # デフォルトで「yes」を返す
        
    return response

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def simple_agent1(obs, cfg):
    # if agent is guesser and turnType is "ask"
    if obs.turnType == "ask":
        response_list = [
            'Is it in Africa?',
            'Is it in America?',
            'Is it in Asia?',
            'Is it in Oceania?',
            'Is it in Eastern Europe?',
            'Is it in Northern Europe?',
            'Is it in Southern Europe?',
            'Is it in Western Europe?',
            'Is it Japan?'
        ]
#         response = response_list[len(obs.questions)]
        response = random.choice(response_list)
    elif obs.turnType == "guess":
        response = "duck"
    elif obs.turnType == "answer":
        response = random.choices(["yes", "no"])[0]
    return response
```

</div>
<div class="column-right">

# 日本語訳

```python
def simple_agent1(obs, cfg):
    # エージェントが推測役でターンタイプが「ask」の場合
    if obs.turnType == "ask":
        response_list = [
            'アフリカにありますか？',
            'アメリカにありますか？',
            'アジアにありますか？',
            'オセアニアにありますか？',
            '東ヨーロッパにありますか？',
            '北ヨーロッパにありますか？',
            '南ヨーロッパにありますか？',
            '西ヨーロッパにありますか？',
            '日本ですか？'
        ]
#         response = response_list[len(obs.questions)]
        response = random.choice(response_list)  # レスポンスリストからランダムに選択
    elif obs.turnType == "guess":  # 推測するターンの場合
        response = "duck"  # 固定で「duck」を返す
    elif obs.turnType == "answer":  # 答えるターンの場合
        response = random.choices(["yes", "no"])[0]  # 「yes」または「no」をランダムに選ぶ
    return response
```

</div>
</details>

In [ ]:
def simple_agent1(obs, cfg):
    # エージェントが推測役でターンタイプが「ask」の場合
    if obs.turnType == "ask":
        response_list = [
            'アフリカにありますか？',
            'アメリカにありますか？',
            'アジアにありますか？',
            'オセアニアにありますか？',
            '東ヨーロッパにありますか？',
            '北ヨーロッパにありますか？',
            '南ヨーロッパにありますか？',
            '西ヨーロッパにありますか？',
            '日本ですか？'
        ]
#         response = response_list[len(obs.questions)]
        response = random.choice(response_list)  # レスポンスリストからランダムに選択
    elif obs.turnType == "guess":  # 推測するターンの場合
        response = "duck"  # 固定で「duck」を返す
    elif obs.turnType == "answer":  # 答えるターンの場合
        response = random.choices(["yes", "no"])[0]  # 「yes」または「no」をランダムに選ぶ
    return response

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#!apt install pigz pv > /dev/null
```

</div>
<div class="column-right">

# 日本語訳

```python
#!apt install pigz pv > /dev/null
```

</div>
</details>

In [ ]:
#!apt install pigz pv > /dev/null

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/transformers/2b-it/2 
```

</div>
<div class="column-right">

# 日本語訳

```python
#!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/transformers/2b-it/2
```

</div>
</details>

In [ ]:
#!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/transformers/2b-it/2

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#%%time

#import random
#from kaggle_environments import make
#agent = "/kaggle/working/submission/main.py"
#env = make("llm_20_questions", debug=True)
#game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])
```

</div>
<div class="column-right">

# 日本語訳

```python
#%%time

#import random
#from kaggle_environments import make
#agent = "/kaggle/working/submission/main.py"
#env = make("llm_20_questions", debug=True)
#game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])
```

</div>
</details>

In [ ]:
#%%time

#import random
#from kaggle_environments import make
#agent = "/kaggle/working/submission/main.py"
#env = make("llm_20_questions", debug=True)
#game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#env.render(mode="ipython", width=600, height=500)
```

</div>
<div class="column-right">

# 日本語訳

```python
#env.render(mode="ipython", width=600, height=500)
```

</div>
</details>

In [ ]:
#env.render(mode="ipython", width=600, height=500)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
```

</div>
<div class="column-right">

# 日本語訳

```python
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
```

</div>
</details>

In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"
        self.turnType = "ask"
        self.keyword = "Japan"
        self.category = "country"
        self.questions = []
        self.answers = []
        self.guesses = []
        
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])

    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)
```

</div>
<div class="column-right">

# 日本語訳

```python
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"
        self.turnType = "ask"
        self.keyword = "Japan"
        self.category = "country"
        self.questions = []
        self.answers = []
        self.guesses = []
        
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSON形式のデータを読み込み

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替キーワードも追加

    # データフレームを作成
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list  # キーワードを追加
    data_pd['category'] = category_list  # カテゴリを追加
    data_pd['alts'] = alts_list  # 代替キーワードを追加
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)  # キーワードデータフレームを作成
```

</div>
</details>

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"
        self.turnType = "ask"
        self.keyword = "Japan"
        self.category = "country"
        self.questions = []
        self.answers = []
        self.guesses = []
        
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSON形式のデータを読み込み

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替キーワードも追加

    # データフレームを作成
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list  # キーワードを追加
    data_pd['category'] = category_list  # カテゴリを追加
    data_pd['alts'] = alts_list  # 代替キーワードを追加
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)  # キーワードデータフレームを作成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
obs = Observation()
cfg = "_"

sample_df = keywords_df.sample()
obs.keyword = sample_df["keyword"].values[0]
obs.category = sample_df["category"].values[0]
alts_list = sample_df["alts"].values[0]
alts_list.append(obs.keyword)

print(f"keyword:{obs.keyword}")

for round in range(20):
    obs.step = round+1
    
    obs.role = "guesser"
    obs.turnType = "ask"
    question = agent(obs, cfg)
    obs.questions.append(question)
    
    obs.role = "answerer"
    obs.turnType = "answer"
    answer = agent(obs, cfg)
    obs.answers.append(answer)
    
    obs.role = "guesser"
    obs.turnType = "guess"
    guess = agent(obs, cfg)
    obs.guesses.append(guess)
    
    print(f"round: {round+1}")
    print(f"question: {question}")
    print(f"answer: {answer}")
    print(f"guess: {guess}")
    
    if guess in alts_list:
        print("Win!!")
        break
```

</div>
<div class="column-right">

# 日本語訳

```python
obs = Observation()  # 観察オブジェクトを作成
cfg = "_"

sample_df = keywords_df.sample()  # キーワードデータフレームからサンプルを抽出
obs.keyword = sample_df["keyword"].values[0]  # キーワードを設定
obs.category = sample_df["category"].values[0]  # カテゴリを設定
alts_list = sample_df["alts"].values[0]  # 代替キーワードを取得
alts_list.append(obs.keyword)  # 代替キーワードリストに選ばれたキーワードを追加

print(f"キーワード: {obs.keyword}")  # キーワードを表示

for round in range(20):  # ゲームのラウンドを20まで繰り返す
    obs.step = round + 1  # ラウンド数を設定
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "ask"  # 質問するターン
    question = agent(obs, cfg)  # 質問をエージェントに送信
    obs.questions.append(question)  # 質問を記録
    
    obs.role = "answerer"  # 答える役の設定
    obs.turnType = "answer"  # 答えるターン
    answer = agent(obs, cfg)  # 答えをエージェントに送信
    obs.answers.append(answer)  # 答えを記録
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "guess"  # 推測するターン
    guess = agent(obs, cfg)  # 推測をエージェントに送信
    obs.guesses.append(guess)  # 推測を記録
    
    print(f"ラウンド: {round + 1}")  # ラウンド番号を表示
    print(f"質問: {question}")  # 質問を表示
    print(f"答え: {answer}")  # 答えを表示
    print(f"推測: {guess}")  # 推測を表示
    
    if guess in alts_list:  # 推測が代替キーワードリストに含まれている場合
        print("勝利!!")  # 勝利のメッセージを表示
        break
```

</div>
</details>

In [ ]:
obs = Observation()  # 観察オブジェクトを作成
cfg = "_"

sample_df = keywords_df.sample()  # キーワードデータフレームからサンプルを抽出
obs.keyword = sample_df["keyword"].values[0]  # キーワードを設定
obs.category = sample_df["category"].values[0]  # カテゴリを設定
alts_list = sample_df["alts"].values[0]  # 代替キーワードを取得
alts_list.append(obs.keyword)  # 代替キーワードリストに選ばれたキーワードを追加

print(f"キーワード: {obs.keyword}")  # キーワードを表示

for round in range(20):  # ゲームのラウンドを20まで繰り返す
    obs.step = round + 1  # ラウンド数を設定
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "ask"  # 質問するターン
    question = agent(obs, cfg)  # 質問をエージェントに送信
    obs.questions.append(question)  # 質問を記録
    
    obs.role = "answerer"  # 答える役の設定
    obs.turnType = "answer"  # 答えるターン
    answer = agent(obs, cfg)  # 答えをエージェントに送信
    obs.answers.append(answer)  # 答えを記録
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "guess"  # 推測するターン
    guess = agent(obs, cfg)  # 推測をエージェントに送信
    obs.guesses.append(guess)  # 推測を記録
    
    print(f"ラウンド: {round + 1}")  # ラウンド番号を表示
    print(f"質問: {question}")  # 質問を表示
    print(f"答え: {answer}")  # 答えを表示
    print(f"推測: {guess}")  # 推測を表示
    
    if guess in alts_list:  # 推測が代替キーワードリストに含まれている場合
        print("勝利!!")  # 勝利のメッセージを表示
        break

In [ ]:
obs = Observation()  # 観察オブジェクトを作成
cfg = "_"

sample_df = keywords_df.sample()  # キーワードデータフレームからサンプルを抽出
obs.keyword = sample_df["keyword"].values[0]  # キーワードを設定
obs.category = sample_df["category"].values[0]  # カテゴリを設定
alts_list = sample_df["alts"].values[0]  # 代替キーワードを取得
alts_list.append(obs.keyword)  # 代替キーワードリストに選ばれたキーワードを追加

print(f"キーワード: {obs.keyword}")  # キーワードを表示

for round in range(20):  # ゲームのラウンドを20まで繰り返す
    obs.step = round + 1  # ラウンド数を設定
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "ask"  # 質問するターン
    question = agent(obs, cfg)  # 質問をエージェントに送信
    obs.questions.append(question)  # 質問を記録
    
    obs.role = "answerer"  # 答える役の設定
    obs.turnType = "answer"  # 答えるターン
    answer = agent(obs, cfg)  # 答えをエージェントに送信
    obs.answers.append(answer)  # 答えを記録
    
    obs.role = "guesser"  # 推測役の設定
    obs.turnType = "guess"  # 推測するターン
    guess = agent(obs, cfg)  # 推測をエージェントに送信
    obs.guesses.append(guess)  # 推測を記録
    
    print(f"ラウンド: {round + 1}")  # ラウンド番号を表示
    print(f"質問: {question}")  # 質問を表示
    print(f"答え: {answer}")  # 答えを表示
    print(f"推測: {guess}")  # 推測を表示
    
    if guess in alts_list:  # 推測が代替キーワードリストに含まれている場合
        print("勝利!!")  # 勝利のメッセージを表示
        break